## Experimental design and partial factorials

### Generate a partial factorial design using information from the response table shown.

The smallest appropriate partial factorial has 8 trials (i.e., the solution with 8 trials has a perfect D-efficiency score equal to 1, no correlation between the variables (i.e., a determinant of 1), and is balanced (i.e., `trial` and `gift` both occur in 4 trials, `speed` and `power` both occur in 4 trials, and `USD150`, `USD160`, `USD170`, and `USD180` each occur in 2 trials).   

In [1]:
import pandas as pd
import pyrsm as rsm

In [2]:
rsm.__version__ # should be 0.9.26 or higher

# from a terminal run
# %pip install --user pyrsm --upgrade

'0.9.26'

In [3]:
## setup pyrsm for autoreload when you edit code and save
%reload_ext autoreload
%autoreload 2
%aimport pyrsm

In [4]:
# needed so you can use R in a Python notebook
%reload_ext rpy2.ipython

In [5]:
%%R
result <- radiant.design::doe(
  factors = c(
    "price; USD150; USD160; USD170; USD180",
    "message; speed; power",
    "promotion; trial; gift"
  ),
  seed = 1234
)
summary(result, eff = TRUE, part = FALSE, full = FALSE)

Experimental design
# trials for partial factorial: 8 
# trials for full factorial   : 16 
Random seed                   : 1234 

Attributes and levels:
price: USD150, USD160, USD170, USD180 
message: speed, power 
promotion: trial, gift 

Design efficiency:
 Trials D-efficiency Balanced
      6        0.135    FALSE
      7        0.082    FALSE
      8        1.000     TRUE

Partial factorial design correlations:
** Note: Variables are assumed to be ordinal **
          price message promotion
price         1       0         0
message       0       1         0
promotion     0       0         1

Estimable effects from partial factorial design:

  price|USD160
  price|USD170
  price|USD180
  message|power
  promotion|gift
  price|USD160:message|power
  price|USD170:message|power 


We can re-estimate the design, specifically for only 8 trials/profiles.

In [6]:
%%R
result <- radiant.design::doe(
  factors = c(
    "price; USD150; USD160; USD170; USD180",
    "message; speed; power",
    "promotion; trial; gift"
  ),
  trials = 8,
  seed = 1234
)
summary(result, eff = TRUE, part = TRUE, full = TRUE)
readr::write_csv(result$part, file = "bizware-partial-factorial.csv")

Experimental design
# trials for partial factorial: 8 
# trials for full factorial   : 16 
Random seed                   : 1234 

Attributes and levels:
price: USD150, USD160, USD170, USD180 
message: speed, power 
promotion: trial, gift 

Design efficiency:
 Trials D-efficiency Balanced
      8        1.000     TRUE

Partial factorial design correlations:
** Note: Variables are assumed to be ordinal **
          price message promotion
price         1       0         0
message       0       1         0
promotion     0       0         1

Partial factorial design:
 trial  price message promotion
     1 USD150   speed     trial
     4 USD150   power      gift
     5 USD160   speed     trial
     8 USD160   power      gift
    10 USD170   speed      gift
    11 USD170   power     trial
    14 USD180   speed      gift
    15 USD180   power     trial

Estimable effects from partial factorial design:

  price|USD160
  price|USD170
  price|USD180
  message|power
  promotion|gift
  price|USD16

Recall that multiple partial factorials may exist that "solve" the experimental design problem.

## Estimate a logistic regression based on the response table shown and predict response for all profiles

Open the `bizware.xls` file. After loading the data into Jupyter we first need to create a _positive_ and _negative response_ variables. Then we can `melt` the data and estimate the logistic regression using (frequency) weights. As stated in the Excel file, assume the sample size for each cell was 2,000


Load information from **data/bizware.xls**:

In [7]:
bizware = pd.read_excel("data/bizware.xls", nrows=8)
bizware.dtypes
bizware

,price,message,promotion,response
0,USD150,speed,trial,0.14
1,USD150,power,gift,0.40
2,USD160,power,trial,0.09
3,USD160,speed,gift,0.13
4,USD170,power,trial,0.06
5,USD170,speed,gift,0.10
6,USD180,speed,trial,0.01
7,USD180,power,gift,0.07


In [8]:
evar = ["price", "message", "promotion"]
bizware[evar] = bizware[evar].astype("category")
bizware["yes"] = 2000 * bizware.response
bizware["no"] = 2000 - bizware.yes
bizware

,price,message,promotion,response,yes,no
0,USD150,speed,trial,0.14,280.0,1720.0
1,USD150,power,gift,0.40,800.0,1200.0
2,USD160,power,trial,0.09,180.0,1820.0
3,USD160,speed,gift,0.13,260.0,1740.0
4,USD170,power,trial,0.06,120.0,1880.0
5,USD170,speed,gift,0.10,200.0,1800.0
6,USD180,speed,trial,0.01,20.0,1980.0
7,USD180,power,gift,0.07,140.0,1860.0


In [9]:
bizware_melt = pd.melt(
    bizware, id_vars=evar, value_vars=["yes", "no"], var_name="resp", value_name="freq",
)

# bizware_melt["resp_yes"] = rsm.ifelse(bizware_melt.resp == "yes", 1, 0)
bizware_melt

,price,message,promotion,resp,freq
0,USD150,speed,trial,yes,280.0
1,USD150,power,gift,yes,800.0
2,USD160,power,trial,yes,180.0
3,USD160,speed,gift,yes,260.0
4,USD170,power,trial,yes,120.0
5,USD170,speed,gift,yes,200.0
6,USD180,speed,trial,yes,20.0
7,USD180,power,gift,yes,140.0
8,USD150,speed,trial,no,1720.0
9,USD150,power,gift,no,1200.0


The results from the logistic regression models are given below.

In [10]:
# form = "resp_yes ~ " + " + ".join(evar)
# form

In [11]:
lr = rsm.model.logistic(
    {"bizwared_melt": bizware_melt},
    rvar="resp",
    lev="yes",
    evar=["price", "message", "promotion"],
    weights="freq",
)
lr.summary()

Logistic regression (GLM)
Data                 : bizwared_melt
Response variable    : resp
Level                : yes
Explanatory variables: price, message, promotion
Weights used         : freq
Null hyp.: There is no effect of x on resp
Alt. hyp.: There is an effect of x on resp

                     OR     OR%  coefficient  std.error  z.value p.value     
Intercept         0.682  -31.8%        -0.38      0.045   -8.593  < .001  ***
price[USD160]     0.371  -62.9%        -0.99      0.065  -15.310  < .001  ***
price[USD170]     0.261  -73.9%        -1.34      0.071  -18.935  < .001  ***
price[USD180]     0.101  -89.9%        -2.29      0.090  -25.463  < .001  ***
message[speed]    0.605  -39.5%        -0.50      0.054   -9.344  < .001  ***
promotion[trial]  0.377  -62.3%        -0.98      0.054  -18.129  < .001  ***

Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Pseudo R-squared (McFadden): 0.127
Pseudo R-squared (McFadden adjusted): 0.127
Area under the RO Curve (AUC

The prediction below only produces the 'partial' results. Check if you can see the difference between these predictions and those from the predictions further below.

In [12]:
bizware_melt["prediction_partial"] = lr.predict(bizware_melt)["prediction"]
bizware_melt

,price,message,promotion,resp,freq,prediction_partial
0,USD150,speed,trial,yes,280.0,0.134524
1,USD150,power,gift,yes,800.0,0.405476
2,USD160,power,trial,yes,180.0,0.087153
3,USD160,speed,gift,yes,260.0,0.132847
4,USD170,power,trial,yes,120.0,0.062847
5,USD170,speed,gift,yes,200.0,0.097153
6,USD180,speed,trial,yes,20.0,0.015476
7,USD180,power,gift,yes,140.0,0.064524
8,USD150,speed,trial,no,1720.0,0.134524
9,USD150,power,gift,no,1200.0,0.405476


The easiest way to generate predictions for all possible profiles (trials) if to use the `levels_list` and `expand_grid` functions in pyrsm. Use the logistic regression object created above and use the newly created dataset to generate the desired predictions

In [13]:
dct = rsm.levels_list(bizware_melt[evar])
dct

{'price': ['USD150', 'USD160', 'USD170', 'USD180'],
 'message': ['speed', 'power'],
 'promotion': ['trial', 'gift']}

In [14]:
bizware_expand = rsm.expand_grid(dct)
bizware_expand

,price,message,promotion
0,USD150,speed,trial
1,USD150,speed,gift
2,USD150,power,trial
3,USD150,power,gift
4,USD160,speed,trial
5,USD160,speed,gift
6,USD160,power,trial
7,USD160,power,gift
8,USD170,speed,trial
9,USD170,speed,gift


In [15]:
bizware_expand["prediction_all"] = lr.predict(bizware_expand)["prediction"]
bizware_expand.sort_values("prediction_all")

,price,message,promotion,prediction_all
12,USD180,speed,trial,0.015476
14,USD180,power,trial,0.025336
8,USD170,speed,trial,0.038973
13,USD180,speed,gift,0.040041
4,USD160,speed,trial,0.054584
10,USD170,power,trial,0.062847
15,USD180,power,gift,0.064524
6,USD160,power,trial,0.087153
9,USD170,speed,gift,0.097153
5,USD160,speed,gift,0.132847


See also the following tutorial video https://youtu.be/lk3ufN2igOo